In [3]:
import sys
#notification导表到notification_etl_s3
def install():
    sc.uninstall_package('boto3')
    sc.install_pypi_package("boto3")
install()
from concurrent import futures
import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime,timedelta
import pytz
import boto3
import json
import pytz
import hmac
import pyspark.sql.functions as F
from hashlib import sha256 
from pyspark.sql.functions import col
from pyspark.sql import types as T
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
today=(datetime.now()+ timedelta(-1)).strftime( '%Y-%m-%d')
yesterday=(datetime.now(pytz.timezone("Asia/Shanghai"))+ timedelta(-1)).strftime( '%Y-%m-%d')
# "arkham_stream_etl":"arkham""protoss_stream_etl":"protoss","credinex_account_stream_etl":"credinex_account","credinex_repeater_stream_etl":"credinex_repeater","credinex_hive_stream_etl":"credinex_hive"
tablemap={"coupon_stream_etl":"coupon"} 
# ,"banda_stream_etl":"1"
tablesql_1=" FROM  (   SELECT    *  , row_number() OVER (PARTITION BY id  ORDER BY date_format(etldate,'yyyy-MM-dd HH:mm:ss.SSS') DESC  , if(etlindex is null ,0,etlindex)  desc)  row_num FROM  "
tablesql_2=" ORDER BY id  ASC )  a  WHERE a.row_num = 1   AND a.kind <> 'delete' "
# 替换手机号
def get_secret_obj():
    df=spark.read.text("s3://rupiahplus-configs/etl/data_secrt/col.json").collect()
    keymap=''
    for  row in df:
        keymap=keymap+row['value']
    json_content=json.loads(keymap)
    return json_content
colmap= get_secret_obj()
print(colmap)
mobileType=colmap["mobileType"]
def getRelmobile(colNm,str):
    if colNm in mobileType:
        relmobile = str.strip().replace("+", "").replace("-", "").replace(" ", "").replace('"', '')
        if (relmobile.startswith("62")):
            relmobile = relmobile.replace("62", "0",1)
        if relmobile.startswith("0") == False:
            relmobile = "0" + relmobile;
        return relmobile
    return str
# hmac_256加密
def hmac_sha256(key,colNm,value):
    if value!=None:
#         处理手机号
        rtn=getRelmobile(colNm,value)
        h = hmac.new(key.encode('utf-8'),digestmod=sha256)
        h.update(rtn.encode('utf-8'))
        h_str = h.hexdigest()
        return h_str
spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
def getTableColum(b):
    colum=""
    for index in range(len(b)):
        if(index>0 and index<len(b)-8):   
            colum=colum+setDef(b[index]["data_type"],b[index]["col_name"])+", "
    return colum[0:len(colum)-2]
def setDef(type,table_col):
    if(type[:7] == 'decimal'):
        return "ifnull("+table_col+",0) as "+table_col
    else:
        return table_col
def add_secret_col(dbType,tableNm,col):
    print(dbType,tableNm)
    secret_Col=[]
    if colmap.get(dbType)!=None and colmap[dbType]!=None and colmap[dbType].get(tableNm)!=None:
        secret_Col=colmap[tablemap[databaseName]][tableNm]
    sh256Col =col
    hmac_key=colmap["hmac_key"]
#     sql="select hmac_sha256('{hmac_key}','123456789') as sh2".format(hmac_key=hmac_key)
    for i in secret_Col:
        sh256Col=sh256Col+ ", hmac_sha256('{hmac_key}','"+i+"', "+i+") "+ i+"_x"
    return  sh256Col.format(hmac_key=hmac_key)
def make_col_temptable(databaseName,row):
    tableName=row["tableName"]
    tablecolum="desc " +databaseName+"."+tableName;
    tableSchema= spark.sql(tablecolum).collect()
    tablepath="s3://rupiahplus-data-warehouse/aliyun/"+tablemap[databaseName]+"/"+row["tableName"]
    spark.catalog.refreshTable(databaseName+"."+tableName)
    secret_sql=add_secret_col(tablemap[databaseName],tableName,getTableColum(tableSchema))+", year"
    sql="select "+secret_sql+ tablesql_1+databaseName+"."+tableName+tablesql_2
    df=spark.sql(sql).drop("etlindex")
    if colmap.get(tablemap[databaseName]) and colmap[tablemap[databaseName]].get(tableName)!=None :
        for i in colmap[tablemap[databaseName]][tableName]:
            df=df.withColumn("col_x",F.col(i+"_x")).withColumn(i+"_x",F.col(i)).withColumn(i,F.col("col_x"))
    df.drop("col_x").write.mode("overwrite").partitionBy("year").orc(tablepath)
if __name__ == "builtins":
#     注册udf
    spark.udf.register("hmac_sha256",hmac_sha256,T.StringType())
    spark = SparkSession\
        .builder\
        .appName("Python Demo")\
        .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .enableHiveSupport()\
        .getOrCreate()
#     spark.conf.set("spark.scheduler.mode","FAIR")
    spark.conf.set("spark.sql.shuffle.partitions",20)
    for databaseName in tablemap:
        print(databaseName)
        databasesql="show tables in "+databaseName
        tables=spark.sql(databasesql)
        tablelist=tables.collect();
        executor=None
        with futures.ThreadPoolExecutor(max_workers=10) as executor:
            futures_result=futures.wait([executor.submit(make_col_temptable,databaseName, tableNm) for tableNm in tablelist])
            for  future in futures_result[0]:
                print(future.result())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-3:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'





{'hmac_key': 'xUIAZ3grtPOxaPNK', 'mobileType': ['phone', 'phone_number', 'user_mobile', 'number', 'caller', 'callee', 'to', 'mobile_phone', 'mobile', 'backup_mobile'], 'extraColumn': {'banda': {'t_personal_info': ['cast(substring(credential_no_x, 11, 2) AS bigint) as birth_year', 'CASE WHEN cast(substring(credential_no_x, 11, 2) AS bigint) > 37 THEN (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 1900) ELSE (year(current_date()) - cast(substring(credential_no_x, 11, 2) AS bigint) - 2000) END as age ']}}, 'luzon': {'t_auto_review_loan': ['ktp', 'mobile'], 't_bankcard': ['card_no'], 't_contact': ['mobile', 'credential_no'], 't_customer': ['mobile', 'imei'], 't_loan_app': ['credential_no'], 't_personal_info': ['credential_no', 'email'], 't_record_contact': ['mobile'], 't_record_personal_info': ['credential_no']}, 'banda': {'t_auto_review_loan': ['ktp', 'mobile'], 't_ai_rudder_schedule_mobile': ['mobile'], 't_mobile_tag_rel': ['mobile'], 't_code': ['send_to'],